# Import Modules

In [ ]:
import pickle as pk
from tqdm import tqdm
import boto3
import pandas as pd
from scipy import io
import numpy as np
import os
import sqlalchemy as sqla
import yaml
import random
import string
from itertools import permutations
import time

# Declare variables

In [ ]:
labelDict= {"H": 'healthy', "I": 'inner race', "O": 'outer race', "B": 'ball', "C": "combination"}
condDict= {"A": 'increasing speed', "B": 'decreasing speed', "C": 'increasing then decreasing speed', 
           "D": 'decreasing then increasing speed'}
columnsOrder= ['instance', 'condition', 'label', 'shaft speed', 'vibration velocity']
nRows = 2 * 10**6

In [ ]:
os.environ['AWS_SHARED_CREDENTIALS_FILE'] = r'../.aws/bearingDefectLocator/credentials'
os.environ['AWS_CONFIG_FILE'] = r'../.aws/bearingDefectLocator/config'
os.environ['DB_CREDENTIALS_FILE'] = r'/home/mattjkenney/projects/.aws/bearingDefectLocator/db.yaml'

In [ ]:
# Get database engine URL
with open(os.environ['DB_CREDENTIALS_FILE'], 'r') as filehandle:
    file = yaml.safe_load(filehandle)

engURL = sqla.URL.create(
    drivername= "postgresql+psycopg2",
    username= file.get('master_username'),
    password= file.get('master_password'),
    port= 5432,
    host= file.get('endpoint'),
    database="bearingvibrations"     
    )
print(engURL)

In [ ]:
# get keys for s3 objects - stored in pickle file
with open('vibs.pk', 'rb') as filehandle:
    keyDict = pk.load(filehandle)
keyLists = list(keyDict.values())
keys = keyLists[0]
for k in keyLists[1:]:
    keys.extend(k)

assert len(keys) == 60 # there should be 60 keys
assert [s[-3:] for s in keys].count("mat") == 60 # all keys should end with "mat"

In [ ]:
# function for building master dataframe
def get_dataframe_from_label(key):

    # verify key indentifiers are valid
    assert key[-9] in labelDict.keys()
    assert key[-7] in condDict.keys()
    assert int(key[-5]) in (1,2,3)

    # download file from s3
    client = boto3.client('s3')
    temp_file = 'temp_file.mat'
    with open(temp_file, 'wb') as data:
        client.download_fileobj('bearingvibrations', key, data)
    data.close()

    # Load matfile and initilize dataframe
    matfile = io.loadmat(temp_file)
    dfn = pd.DataFrame()

    # Load dataframe with data
    dfn['vibration velocity'] = tuple(np.reshape(matfile.get('Channel_1'), (nRows,)))
    dfn['shaft speed'] = tuple(np.reshape(matfile.get('Channel_2'), (nRows,)))
    dfn['label'] = tuple([labelDict.get(key[-9]) for i in range(nRows)])
    dfn['condition'] = tuple([condDict.get(key[-7]) for i in range(nRows)])
    dfn['instance'] = tuple([str(key[-5]) for i in range(nRows)])

    # Re-organize columns
    dfn = dfn.reindex(columns= columnsOrder)

    # Final data validations
    assert dfn.shape == (2000000, 5) # each dataframe instance should have 2 million rows and 5 columns

    # remove temporary file
    os.remove(temp_file)

    return dfn

# Get Data from S3

In [ ]:
# get list of dataframes for all labels
dfs = []
for i in tqdm(keys):
    df = get_dataframe_from_label(i)
    assert df.shape == (2000000, 5) # validates shape
    assert list(df.columns) == columnsOrder # validates columns and column order
    dfs.append(df)

In [ ]:
# visual inspection of sample dataframe
dfs[0].info()

In [ ]:
bts = sum([df.memory_usage(deep=True).sum() for df in dfs])
mbs = bts / 1000
gbs = mbs / 1000
tbs = gbs / 1000
print(gbs, 'GB')
print(tbs, 'TB')

In [ ]:
gbs * 0.115

# Push Data to RDS

## Get DB Engine

In [ ]:
# Get database engine   
engine = sqla.create_engine(engURL)
connection = engine.connect()
print(connection)

## Create Table in RDS

In [ ]:
for df in tqdm(dfs):
    df.to_sql("bearing_vibrations", connection, if_exists='append')